In [ ]:
import os

import numpy as np 
import pandas as pd 
import json

import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as mpimg


train_image_path = "/kaggle/input/cassava-leaf-disease-classification/train_images/"
test_image_path = "/kaggle/input/cassava-leaf-disease-classification/test_images/"

In [ ]:
traindf = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
traindf.head()

In [ ]:
traindf.shape

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = traindf['image_id'][0:12]

for i, img_path in enumerate(next_pix):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')

    img = mpimg.imread(train_image_path + img_path)
    plt.imshow(img)

plt.show()

Referecnce : https://www.kaggle.com/bulentsiyah/learn-opencv-by-examples-with-python

# Sharpening Image

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = traindf['image_id'][0:12]

for i, img_path in enumerate(next_pix):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')
    
    img = cv2.imread(train_image_path + img_path,1)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Create our shapening kernel, we don't normalize since the 
  
    kernel_sharpening = np.array([[1,2,3], 
                                  [4,-33,5], 
                                  [6,7,8]])

    # applying different kernels to the input image
    sharpened = cv2.filter2D(image, -1, kernel_sharpening)

    plt.imshow(sharpened)

plt.show()




# Thresholding Image

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = traindf['image_id'][0:12]

for i, img_path in enumerate(next_pix):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')
    
    img = cv2.imread(train_image_path + img_path,0)
    
    # Values below 100 goes to 0 (black), everything above goes to 200 (white)
    ret,thresh1 = cv2.threshold(img, 100, 200, cv2.THRESH_BINARY)


    plt.imshow(thresh1)

plt.show()

# Binarization 

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = traindf['image_id'][0:12]

for i, img_path in enumerate(next_pix):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')
    
    img = cv2.imread(train_image_path + img_path,0)
    
    # It's good practice to blur images as it removes noise
    image = cv2.GaussianBlur(img, (3,3), 0)



    plt.imshow(image)

plt.show()

# Adaptive Thresholding

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = traindf['image_id'][0:12]

for i, img_path in enumerate(next_pix):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')
    
    img = cv2.imread(train_image_path + img_path,0)
    
    # It's good practice to blur images as it removes noise
    image = cv2.GaussianBlur(img, (3,3), 0)

    # Using adaptiveThreshold
    thresh = cv2.adaptiveThreshold(image, 200, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 3)

    plt.imshow(thresh)

plt.show()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = traindf['image_id'][0:12]

for i, img_path in enumerate(next_pix):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')
    
    img = cv2.imread(train_image_path + img_path,0)
    
    # It's good practice to blur images as it removes noise
    image = cv2.GaussianBlur(img, (3,3), 0)

    _, th2 = cv2.threshold(image, 0, 200, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    plt.imshow(th2)

plt.show()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = traindf['image_id'][0:12]

for i, img_path in enumerate(next_pix):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')
    
    img = cv2.imread(train_image_path + img_path,0)
    
    # Otsu's thresholding after Gaussian filtering
    blur = cv2.GaussianBlur(img, (5,5), 0)
    
    _, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    plt.imshow(th3)

plt.show()

# Background Removal

In [ ]:
f = open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json","r")
data = json.load(f)
data['0']

In [ ]:
X_data = []
Y_data = []

for i in range(0,600):
    img = load_img(train_image_path + traindf['image_id'][i])
    X_data.append(img_to_array(img))
    Y_data.append(traindf['label'][i])

In [ ]:
X_data = np.array(X_data)

In [ ]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0])
Y_data = to_categorical(Y_data, num_classes = 5)
Y_data.shape

In [ ]:
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size = 0.2, random_state=45)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (600,800,3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(5, activation = "softmax"))

In [ ]:
# Compile the model
model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["acc"])

In [ ]:
epochs = 50
batch_size = 8

In [ ]:
#history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, 
#          validation_data = (X_val, Y_val), verbose = 2)

In [ ]:
datagen = ImageDataGenerator(
        rescale=1./255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        shear_range=0.1,
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,
        fill_mode='nearest')  # randomly flip images


datagen.fit(X_train)

In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, 
                              validation_data = (X_val,Y_val),
                              verbose = 1, 
                              steps_per_epoch=4)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()